In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon

In [7]:
#import folium

In [8]:
# m = folium.Map(location=(38.029, -78.477), zoom_start=13)

# geojson_data = requests.get(
#    'https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_1/MapServer/13/query?outFields=*&where=1%3D1&f=geojson'
# ).json()

# folium.GeoJson(
#     geojson_data,
#     style_function=lambda feature: {
#         "fillColor": "blue",
#     },
# ).add_to(m)

# folium.LayerControl().add_to(m)

# schools = "Elementary_School_Zone_Area.geojson"
# folium.GeoJson(schools,  style_function=lambda feature: {
#         "color": "red",
#     },).add_to(m)

# m

In [9]:
#Load in a GeoJSON of Charlottesville census blocks
data = gpd.read_file('https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_1/MapServer/13/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [10]:
#Load in a GeoJSON of the elementary school zone boundaries
schools = gpd.read_file('https://gisweb.charlottesville.org/arcgis/rest/services/OpenData_1/MapServer/16/query?outFields=*&where=1%3D1&f=geojson')
schools

,OBJECTID,ZONE,geometry
0,2,Clark,"POLYGON ((-78.47802 38.02863, -78.48150 38.029..."
1,4,Jackson-Via,"POLYGON ((-78.49029 38.02682, -78.49203 38.027..."
2,5,Johnson,"POLYGON ((-78.49289 38.02676, -78.49275 38.026..."
3,6,Greenbrier,"MULTIPOLYGON (((-78.47221 38.06803, -78.47100 ..."
4,1,Burnley-Moran,"POLYGON ((-78.49375 38.03334, -78.49380 38.033..."
5,3,Venable,"POLYGON ((-78.47841 38.04517, -78.47901 38.045..."


In [11]:
#Set variables for each school's boundaries
clark = schools['geometry'][0]
jv = schools['geometry'][1]
johnson = schools['geometry'][2]
greenbrier = schools['geometry'][3]
bm = schools['geometry'][4]
venable = schools['geometry'][5]


In [12]:
#Write a function that will label each block by its assigned school
def find_school_zone(schools, data):
    for x in range(803):
        if clark.contains(data['geometry'][x])==True:
            data['school_zone'][x]="Clark",
        elif jv.contains(data['geometry'][x])==True:
            data['school_zone'][x]="Jackson-Via",
        elif johnson.contains(data['geometry'][x])==True:
            data['school_zone'][x]="Johnson",
        elif greenbrier.contains(data['geometry'][x])==True:
            data['school_zone'][x]="Greenbrier",
        elif bm.contains(data['geometry'][x])==True:
            data['school_zone'][x]="Burnley-Moran",
        elif venable.contains(data['geometry'][x])==True:
            data['school_zone'][x]="Venable",
        else: data['school_zone'][x]="Manual"

In [14]:
data['school_zone']="Manual"

In [15]:
#Run the function
pd.options.mode.chained_assignment = None
find_school_zone(schools, data)

In [16]:
#Check counts
data['school_zone'].value_counts()

Manual              196
(Venable,)          189
(Burnley-Moran,)    175
(Clark,)             78
(Greenbrier,)        66
(Jackson-Via,)       53
(Johnson,)           46
Name: school_zone, dtype: int64

In [17]:
#The function missed about 25% of zones, largely those on borders. Since the data set is rather small, I will
#refer to the visual map and input these school zones manually. In cases of a block that covers multiple attendance
#areas, I will assign to the school that has the clear majority of the zone. If a clear majority is not obvious,
#I will label it "Split".
manuals = data.loc[data['school_zone']=='Manual']

In [18]:
manuals

,OBJECTID,ID,cvgis.CITY.census_block_2010_area.AREA,Coloring,Block,BlockGroup,Tract,County,State,Block2000,...,NH18__DOJ_Asn,NH18__DOJ_Hwn,NH18__DOJ_Oth,NH18__DOJ_OthMR,Housing_Units,HU_Occupied,HU_Vacant,Sliver,geometry,school_zone
0,1,11532,NaN,1,515400006002011,515400006002,51540000600,51540,51,515400006002011,...,150,0,0,5,156,148,8,0,"POLYGON ((-78.51184 38.02624, -78.51499 38.025...",Manual
2,3,11570,NaN,0,515400006002006,515400006002,51540000600,51540,51,515400006002006,...,0,0,0,0,2,2,0,0,"POLYGON ((-78.51974 38.02846, -78.51971 38.028...",Manual
4,5,11604,NaN,1,515400006002004,515400006002,51540000600,51540,51,515400006002004,...,58,0,0,1,0,0,0,0,"POLYGON ((-78.51939 38.02846, -78.51974 38.028...",Manual
5,6,11624,NaN,0,515400006002003,515400006002,51540000600,51540,51,515400006002003,...,11,0,0,0,19,17,2,0,"POLYGON ((-78.51650 38.02864, -78.51649 38.027...",Manual
6,7,11645,NaN,1,515400006002002,515400006002,51540000600,51540,51,515400006002002,...,198,1,0,3,178,173,5,0,"POLYGON ((-78.51447 38.02871, -78.51435 38.028...",Manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,794,21144,NaN,1,515400006001003,515400006001,51540000600,51540,51,515400006001002,...,0,0,0,0,0,0,0,0,"POLYGON ((-78.49868 38.03031, -78.49887 38.030...",Manual
796,797,21200,NaN,3,515400002022009,515400002022,51540000202,51540,51,515400002022019,...,0,0,0,0,0,0,0,0,"POLYGON ((-78.49735 38.03377, -78.49754 38.033...",Manual
797,798,21218,NaN,1,515400002021030,515400002021,51540000202,51540,51,515400002021020,...,0,0,0,0,0,0,0,0,"POLYGON ((-78.49651 38.03175, -78.49655 38.031...",Manual
798,799,21238,NaN,0,515400006001001,515400006001,51540000600,51540,51,515400006001000,...,0,0,0,0,0,0,0,0,"POLYGON ((-78.49539 38.03139, -78.49547 38.031...",Manual


In [19]:
#Inputs generated by creating an ArcGIS map of the two datasets, accessed through opendata.charlottesville.org
data['school_zone'][0]='Venable'
data['school_zone'][2]='Venable'
data['school_zone'][4]='Venable'
data['school_zone'][5]='Venable'
data['school_zone'][6]='Venable'
data['school_zone'][7]='Venable'
data['school_zone'][8]='Venable'
data['school_zone'][9]='Venable'
data['school_zone'][13]='Venable'
data['school_zone'][17]='Venable'
data['school_zone'][19]='Venable'
data['school_zone'][20]='Venable'
data['school_zone'][21]='Venable'
data['school_zone'][24]='Jackson-Via'
data['school_zone'][27]='Jackson-Via'
data['school_zone'][34]='Johnson'
data['school_zone'][40]='Johnson'
data['school_zone'][42]='Jackson-Via'
data['school_zone'][47]='Split'
data['school_zone'][49]='Jackson-Via'
data['school_zone'][52]='Jackson-Via'
data['school_zone'][60]='Johnson'
data['school_zone'][62]='Jackson-Via'
data['school_zone'][65]='Johnson'
data['school_zone'][68]='Jackson-Via'
data['school_zone'][70]='Jackson-Via'
data['school_zone'][74]='Jackson-Via'
data['school_zone'][77]='Venable'
data['school_zone'][78]='Johnson'
data['school_zone'][79]='Venable'
data['school_zone'][83]='Johnson'
data['school_zone'][84]='Split'
data['school_zone'][87]='Johnson'
data['school_zone'][99]='Venable'
data['school_zone'][108]='Venable'
data['school_zone'][110]='Venable'
data['school_zone'][111]='Venable'
data['school_zone'][113]='Venable'
data['school_zone'][115]='Venable'
data['school_zone'][116]='Venable'
data['school_zone'][117]='Venable'
data['school_zone'][123]='Venable'
data['school_zone'][125]='Venable'
data['school_zone'][126]='Venable'
data['school_zone'][127]='Venable'
data['school_zone'][128]='Split'
data['school_zone'][129]='Greenbrier'
data['school_zone'][130]='Greenbrier'
data['school_zone'][131]='Venable'
data['school_zone'][132]='Venable'
data['school_zone'][133]='Venable'
data['school_zone'][135]='Venable'
data['school_zone'][158]='Jackson-Via'
data['school_zone'][166]='Johnson'
data['school_zone'][169]='Split'
data['school_zone'][174]='Jackson-Via'
data['school_zone'][176]='Clark'
data['school_zone'][177]='Clark'
data['school_zone'][178]='Clark'
data['school_zone'][179]='Jackson-Via'
data['school_zone'][183]='Jackson-Via'
data['school_zone'][193]='Jackson-Via'
data['school_zone'][196]='Split'
data['school_zone'][197]='Johnson'
data['school_zone'][198]='Johnson'
data['school_zone'][199]='Johnson'
data['school_zone'][202]='Johnson'
data['school_zone'][205]='Johnson'
data['school_zone'][206]='Clark'
data['school_zone'][209]='Clark'
data['school_zone'][210]='Jackson-Via'
data['school_zone'][212]='Clark'
data['school_zone'][214]='Clark'
data['school_zone'][225]='Greenbrier'
data['school_zone'][232]='Burnley-Moran'
data['school_zone'][233]='Burnley-Moran'
data['school_zone'][235]='Burnley-Moran'
data['school_zone'][236]='Burnley-Moran'
data['school_zone'][238]='Burnley-Moran'
data['school_zone'][239]='Burnley-Moran'
data['school_zone'][240]='Greenbrier'
data['school_zone'][246]='Greenbrier'
data['school_zone'][251]='Greenbrier'
data['school_zone'][253]='Greenbrier'
data['school_zone'][255]='Burnley-Moran'
data['school_zone'][256]='Burnley-Moran'
data['school_zone'][257]='Greenbrier'
data['school_zone'][258]='Split'
data['school_zone'][286]='Greenbrier'
data['school_zone'][288]='Split'
data['school_zone'][290]='Split'
data['school_zone'][304]='Greenbrier'
data['school_zone'][306]='Greenbrier'
data['school_zone'][311]='Greenbrier'
data['school_zone'][312]='Greenbrier'
data['school_zone'][313]='Burnley-Moran'
data['school_zone'][314]='Greenbrier'
data['school_zone'][315]='Greenbrier'
data['school_zone'][321]='Greenbrier'
data['school_zone'][344]='Split'
data['school_zone'][345]='Venable'
data['school_zone'][360]='Venable'
data['school_zone'][362]='Greenbrier'
data['school_zone'][366]='Greenbrier'
data['school_zone'][368]='Greenbrier'
data['school_zone'][369]='Venable'
data['school_zone'][371]='Greenbrier'
data['school_zone'][372]='Greenbrier'
data['school_zone'][373]='Venable'
data['school_zone'][377]='Venable'
data['school_zone'][380]='Venable'
data['school_zone'][384]='Venable'
data['school_zone'][385]='Venable'
data['school_zone'][386]='Venable'
data['school_zone'][390]='Burnley-Moran'
data['school_zone'][396]='Burnley-Moran'
data['school_zone'][398]='Burnley-Moran'
data['school_zone'][407]='Burnley-Moran'
data['school_zone'][408]='Burnley-Moran'
data['school_zone'][419]='Greenbrier'
data['school_zone'][421]='Greenbrier'
data['school_zone'][423]='Greenbrier'
data['school_zone'][425]='Greenbrier'
data['school_zone'][431]='Venable'
data['school_zone'][441]='Split'
data['school_zone'][450]='Split'
data['school_zone'][470]='Split'
data['school_zone'][472]='Split'
data['school_zone'][476]='Split'
data['school_zone'][477]='Greenbrier'
data['school_zone'][482]='Greenbrier'
data['school_zone'][484]='Greenbrier'
data['school_zone'][485]='Greenbrier'
data['school_zone'][486]='Greenbrier'
data['school_zone'][487]='Greenbrier'
data['school_zone'][488]='Greenbrier'
data['school_zone'][490]='Greenbrier'
data['school_zone'][493]='Greenbrier'
data['school_zone'][494]='Greenbrier'
data['school_zone'][495]='Greenbrier'
data['school_zone'][497]='Jackson-Via'
data['school_zone'][499]='Jackson-Via'
data['school_zone'][500]='Split'
data['school_zone'][501]='Split'
data['school_zone'][502]='Clark'
data['school_zone'][503]='Clark'
data['school_zone'][505]='Clark'
data['school_zone'][506]='Clark'
data['school_zone'][508]='Clark'
data['school_zone'][511]='Clark'
data['school_zone'][551]='Burnley-Moran'
data['school_zone'][552]='Clark'
data['school_zone'][553]='Clark'
data['school_zone'][554]='Clark'
data['school_zone'][557]='Clark'
data['school_zone'][569]='Clark'
data['school_zone'][571]='Burnley-Moran'
data['school_zone'][573]='Clark'
data['school_zone'][576]='Burnley-Moran'
data['school_zone'][618]='Burnley-Moran'
data['school_zone'][622]='Greenbrier'
data['school_zone'][623]='Greenbrier'
data['school_zone'][627]='Greenbrier'
data['school_zone'][633]='Burnley-Moran'
data['school_zone'][693]='Split'
data['school_zone'][707]='Split'
data['school_zone'][708]='Split'
data['school_zone'][709]='Venable'
data['school_zone'][710]='Burnley-Moran'
data['school_zone'][711]='Venable'
data['school_zone'][713]='Split'
data['school_zone'][715]='Split'
data['school_zone'][716]='Clark'
data['school_zone'][717]='Clark'
data['school_zone'][721]='Burnley-Moran'
data['school_zone'][722]='Burnley-Moran'
data['school_zone'][723]="Burnley-Moran"
data['school_zone'][740]="Clark"
data['school_zone'][742]='Burnley-Moran'
data['school_zone'][744]='Burnley-Moran'
data['school_zone'][745]='Clark'
data['school_zone'][748]='Burnley-Moran'
data['school_zone'][750]='Burnley-Moran'
data['school_zone'][751]='Burnley-Moran'
data['school_zone'][752]='Venable'
data['school_zone'][756]='Venable'
data['school_zone'][762]='Clark'
data['school_zone'][768]='Clark'
data['school_zone'][769]='Burnley-Moran'
data['school_zone'][779]='Johnson'
data['school_zone'][782]='Venable'
data['school_zone'][793]='Venable'
data['school_zone'][796]='Venable'
data['school_zone'][797]='Venable'
data['school_zone'][798]='Venable'
data['school_zone'][802]='Venable'
data['school_zone'][803]='Venable'

In [20]:
data['school_zone'].value_counts()

(Venable,)          189
(Burnley-Moran,)    175
(Clark,)             78
(Greenbrier,)        66
(Jackson-Via,)       53
Venable              51
(Johnson,)           46
Greenbrier           39
Burnley-Moran        29
Clark                25
Split                21
Jackson-Via          17
Johnson              14
Name: school_zone, dtype: int64

In [21]:
#The function inserted school names as tuples, so we need to standardize them
data['school_zone'] = data['school_zone'].apply(lambda x: x if type(x)==str else x[0])

In [22]:
data['school_zone'].value_counts()

Venable          240
Burnley-Moran    204
Greenbrier       105
Clark            103
Jackson-Via       70
Johnson           60
Split             21
Name: school_zone, dtype: int64

In [23]:
#Let's start with dummy models.

#Predict one of the six schools:
one_of_six = 1/6

#Predict the most common school (Venable):
dummy = 240/803

one_of_six, dummy

(0.16666666666666666, 0.298879202988792)

In [24]:
#Done with the shapes, so let's convert back to a standard dataframe
data.pop('geometry')
data=pd.DataFrame(data)

In [25]:
#blocks = pd.read_csv('data_download/regional_atlas_block_groups.csv')

In [26]:
#for col in blocks.columns:
    #print(col)

In [27]:
#blocks.loc[1]()

In [28]:
#The splits are not very helpful, so we'll drop those
data = data[data['school_zone'] != 'Split']

# dic = {'Venable': 1, 'Burnley-Moran': 2, 'Greenbrier': 3, 'Clark': 4, 'Jackson-Via': 5, 'Johnson': 6}
# data= data.replace({'school_zone': dic})

In [29]:
X = data.drop('school_zone', axis=1)
y = data['school_zone']

In [30]:
#Look at the available information
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 782 entries, 0 to 802
Data columns (total 380 columns):
 #   Column                                  Dtype  
---  ------                                  -----  
 0   OBJECTID                                int64  
 1   ID                                      int64  
 2   cvgis.CITY.census_block_2010_area.AREA  float64
 3   Coloring                                int64  
 4   Block                                   object 
 5   BlockGroup                              object 
 6   Tract                                   object 
 7   County                                  object 
 8   State                                   object 
 9   Block2000                               object 
 10  MCD                                     object 
 11  SubMCD                                  object 
 12  Place                                   object 
 13  ConCity                                 object 
 14  VTD                                     o

In [31]:
#Some of the information is irrelevant; some of it is the same for all rows; and some of it is repetitive.
#We will start with a selection of what seem like the most relevant features for our problem.
columns = ['Population', 'White', 'Black', 'AmIndian', 'Asian', 'Hawaiian', 'Other', 'WhtBlk','Housing_Units', 'HU_Occupied']
X = X[columns]

In [32]:
X

,Population,White,Black,AmIndian,Asian,Hawaiian,Other,WhtBlk,Housing_Units,HU_Occupied
0,377,174,33,0,144,0,3,5,156,148
1,0,0,0,0,0,0,0,0,0,0
2,10,8,0,0,0,0,2,0,2,2
3,2,2,0,0,0,0,0,0,1,1
4,195,104,28,0,55,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...
798,0,0,0,0,0,0,0,0,0,0
799,83,76,1,0,6,0,0,0,18,18
800,103,88,0,0,8,0,0,0,23,23
801,1249,1028,73,1,105,1,15,5,447,437


In [33]:
#Engineer a few features and then ensure we aren't dividing by zero.
X['Density'] = X['Population']/X['Housing_Units']
X['Occupancy'] = X['HU_Occupied']/X['Housing_Units']
X=X.replace({float('inf'):0})

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=12)

In [56]:
from sklearn.dummy import DummyClassifier 

dummy= DummyClassifier(strategy='uniform')
random_guess = dummy.fit(X_train, y_train).score(X_test, y_test)

dummy2 = DummyClassifier(strategy='most_frequent')
guessing_venable = dummy2.fit(X_train, y_train).score(X_test, y_test)

random_guess, guessing_venable

(0.1836734693877551, 0.32142857142857145)

In [35]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
pipe = Pipeline(steps=(('scale', StandardScaler()), ('knn', KNNImputer()), ('lr', LogisticRegressionCV(multi_class='multinomial', max_iter=1000))))


In [36]:
le = LabelEncoder()
y_train_labelled = le.fit_transform(y_train)
y_test_labelled = le.fit_transform(y_test)

In [43]:
pipe.fit(X_train, y_train_labelled).score(X_train, y_train_labelled)

0.4044368600682594

In [50]:
from sklearn.tree import DecisionTreeClassifier
pipe2 = Pipeline(steps=(('scale', StandardScaler()), ('knn', KNNImputer()), ('tree', DecisionTreeClassifier(max_depth=6))))

In [51]:
pipe2.fit(X_train, y_train_labelled).score(X_train, y_train_labelled)

0.447098976109215